### 概念

DDPM：Denoising Diffusion Probabilistic Models（去噪扩散概率模型）  
U-Net：一种DDPM模型  
ViT：Vision Transformer  
Dit：Diffusion Transformer  
VAE：Variational Autoencoder 一个用于升维降维的概率模型，把原始图片压缩为潜在空间的特征值分布  
CLIP：Contrastive Language-Image Pretraining（对比语言图像预训练）  
LDM：Latent Diffusion Model  

Autoencoder: 
1. 由encoder和decoder组成，encoder将输入数据转换为潜在空间的表示，decoder则将潜在表示重建为原始数据。然后我们可以对比输入数据和重建数据之间的差异，计算reconstruction loss。这个时候就可以训练了。encoder可以是全连接的网络或者卷积网络。
2. latent vector通常是10-100维的向量。
3. Autoencoder的问题在于，虽然能够压缩输入到潜在空间，但是无法保证潜在空间分布可能存在问题，例如将手写图片2变为9的过程中，中间的值可能生成的一些没有意义的符号。

Denosing Autoencoder：假设我们给一个图片加上了噪声，然后这个图片变成Autoencoder的输入，经过encoder和decoder，来尝试重建加上噪声前的图片（reconstruction loss不是和加上噪声的输入进行计算），这个时候来进行模型训练，最后能够得到一个去噪的模型，模型能够将噪声去掉，恢复原始图片。

Neural Inpainting：可以理解为一种特殊的去噪模型，输入是一个有缺失部分的图片，输出是完整的图片。例如可以用于去除水印等场景。

variational autoencoder：
1. 在Autoencoder的基础上，encoder输出的是一个分布而不是latent vector/variables。普通的bottleneck vecotor被替换为了两个向量，一个是mean vector，一个是standard deviation vector。这个时候decoder的输入是从这两个向量中采样得到的latent vector。这样做的好处是能够让模型学习到数据的分布。
2. 在拿到mean vector和standard deviation vector后，我们有一个sample的操作，然后拿到latent vector，但是这个sample操作是不可微的，所以我们需要使用reparameterization trick来使得这个sample操作可微。具体来说，我们可以将latent vector表示为mean vector加上一个标准差乘以一个随机噪声，这样就可以将sample操作转化为可微的操作。
3. variational autoencoder常用于强化学习领域，因为reinforcement learning有一个特点就是sparse rewards，这个时候需要花费大量的时间来训练模型。这个时候使用VAE作为某种特征提取器，这个时候我们可以运行在representation space上，而不是直接在原始数据上进行训练。
4. vae中核心是reparameterization trick，详细讲解参考：https://snawarhussain.com/blog/genrative%20models/python/vae/tutorial/machine%20learning/Reparameterization-trick-in-VAEs-explained/  
- 我的理解：要引入随机性，但是又要保证可以微分（反向传播），所以将确定的不确定的（添加的随机的部分）分离开，`z=μ+σ∗ε`，`ε`代表随机性

disentangled variational autoencoder：
1. 在variational autoencoder的基础上，encoder输出的是多个分布而不是一个分布。每个分布对应一个潜在变量，这些潜在变量可以被看作是数据的不同方面。
2. 这个时候修改latent vector，都会对输出有一些直观的影响。就像是从不同维度去修改图片的内容一样。
3. disentangled如果的维度太小，代表模型的自由度会降低，无法通用化。

kullback-leibler divergence（KL divergence）：用来衡量两个概率分布之间的差异

VAE模型中，**通道数（channel）的变化**主要发生在卷积层（`nn.Conv2d`）和转置卷积层（`nn.ConvTranspose2d`）中。以下是具体的通道数变化过程及位置说明：

---

### **1. Encoder部分（压缩路径）**
通道数的变化由输入图像（3通道）逐步增加到256通道，最后通过1×1卷积降维到4通道（潜在空间维度）：

| **层**                | **输入通道** → **输出通道** | **变化位置**                     | **输出尺寸（C×H×W）** |
|-----------------------|---------------------------|----------------------------------|----------------------|
| 输入图像               | 3                         | -                                | 3×512×512           |
| `encoder[0]`（第一层） | 3 → 64                    | `_conv_block(in_channels=3, out_channels=64)` | 64×256×256          |
| `encoder[1]`（第二层） | 64 → 128                  | `_conv_block(64, 128)`           | 128×128×128         |
| `encoder[2]`（第三层） | 128 → 256                 | `_conv_block(128, 256)`          | 256×64×64           |
| `fc_mu` / `fc_var`    | 256 → 4                   | `nn.Conv2d(256, latent_dim=4, 1)` | 4×64×64             |

**关键点**：
- 每个`_conv_block`中的`nn.Conv2d`通过`out_channels`参数显式指定输出通道数。
- 最后的`fc_mu`和`fc_var`是1×1卷积，仅改变通道数（256→4），不改变空间尺寸（64×64）。

---

### **2. Decoder部分（解压缩路径）**
通道数的变化与Encoder相反，从潜在空间（4通道）逐步恢复到原始图像通道数（3通道）：

| **层**                     | **输入通道** → **输出通道** | **变化位置**                              | **输出尺寸（C×H×W）** |
|----------------------------|---------------------------|------------------------------------------|----------------------|
| 潜在空间输入（`z`）          | 4                         | -                                        | 4×64×64             |
| `decoder_input`             | 4 → 256                   | `nn.ConvTranspose2d(latent_dim=4, 256, 1)` | 256×64×64           |
| `decoder[0]`（第一层）       | 256 → 128                 | `_conv_transpose_block(256, 128)`        | 128×128×128         |
| `decoder[1]`（第二层）       | 128 → 64                  | `_conv_transpose_block(128, 64)`         | 64×256×256          |
| `decoder[2]`（第三层）       | 64 → 3                    | `_conv_transpose_block(64, in_channels=3)` | 3×512×512           |

### CNN

1. stride为2 ==> 决定了每次conv_block处理完，尺寸会减半  
2. in_channel为3，代表RGB三个通道  
3. out_channel为64，代表64个`3 * 3 * 3`的卷积核，在每个位置都计算一次，得到一个特征值，一个位置能计算出64个值来，所以就是64的out_channel（64个特征维度）。一个卷积核中有权重信息（[3, 3, 3]的维度），和输入图片的RGB三个通道的`3 * 3 = 9`个数，对应位置相乘求和+bias，得到一个卷积核计算出来的特征值，这样的计算过程会有64次。  
4. 前面三多层cnn处理后，图像的大小减半3次，从512 -> 64，特征维度从3到256，然后这个时候再走`nn.Conv2d(256, latent_dim=4, 1)`，输入维度256，输出维度4，卷积核大小为1×1，将输入从 256 维映射到 latent_dim 维（4维）